In [159]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from datetime import datetime

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

In [160]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [161]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [162]:
# Current coordinates of the user
latitude = 52.3667
longitude = 4.8945

# User specifications:
radii = [500, 300] # maximum radius values for 2 searches

keywords = ['taco', 'museum'] # keywords for searches
num_of_results = [3, 2] # number of results that user want to see

In [163]:
CLIENT_ID = 'DJM1ICTW2V55WQKY2URDP0RSLYHOKNWGZOONWSHNTHD3EKUU' # your Foursquare ID
CLIENT_SECRET = 'PMG40DKGUJG4NIY4ITCD3HIMG5N3IAM5QSBM5P4APMI0W15J' # your Foursquare Secret
VERSION = '20180604'

LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&query={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, keywords[0], VERSION, radii[0], LIMIT)

results = requests.get(url).json()

Your credentails:
CLIENT_ID: DJM1ICTW2V55WQKY2URDP0RSLYHOKNWGZOONWSHNTHD3EKUU
CLIENT_SECRET:PMG40DKGUJG4NIY4ITCD3HIMG5N3IAM5QSBM5P4APMI0W15J


In [164]:
def JSON_TO_DF(results, ResultsNumber):
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.location.address', 'venue.location.distance', 'venue.location.lat', 'venue.location.lng', 'venue.id']

    data = dataframe.loc[:, filtered_columns]

    # clean columns
    data.columns = [col.split('.')[-1] for col in data.columns]

    number_of_ratings  = []
    ratings = []

    for i, venue_id in enumerate(data['id']):
        url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result2 = requests.get(url2).json()
    
        try: # If there is no rating, the number of ratings will be filled with NaN
            number_of_ratings.append(result2['response']['venue']['ratingSignals'])
        except:
            number_of_ratings.append(np.nan)
        
        try: # If there is no rating, the rating will be filled with NaN
            ratings.append(result2['response']['venue']['rating'])
        except:
            ratings.append(np.nan)
    
    data['number_of_ratings'] = number_of_ratings # Add the new columns to the dataFrame
    data['ratings'] = ratings

    data.sort_values(by = 'ratings', ascending = False, inplace = True) # Sort data by the ratings in descending order
    data.dropna(inplace = True) # Drop the Nan values
    data.reset_index(drop = True, inplace = True) # Reset the index in ascending order

    if (ResultsNumber < data.shape[0]): # crop the data, so it shows as many venues as client desires
        data = data.iloc[0:ResultsNumber]
        
    return data



In [165]:
data1 = JSON_TO_DF(results,num_of_results[0])
data1.head()

,name,address,distance,lat,lng,id,number_of_ratings,ratings
0,Cannibale Royale,Handboogstraat 17-A,318,52.367946,4.890285,5336fe68498e18d7e65f5803,2157.0,8.9
1,Salsa Shop,Amstelstraat 32A,375,52.366226,4.899969,53c3ca66498ea876ac0e1349,256.0,7.6
2,Escape,Rembrandtplein 11,125,52.366846,4.896326,4a2705e2f964a52032891fe3,671.0,7.2


In [166]:
x = np.arange(data1.shape[0])
ys = [i + x + (i*x)**2 for i in range(data1.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the client location

# add a red circle marker to represent the client
folium.features.RegularPolygonMarker(
    [latitude, longitude],
    number_of_sides=3,
    rotation=30,
    radius=20,
    color='red',
    popup='You are HERE!'
).add_to(venues_map)

i=0
# add the first level search results with different colors of circle markers
for lat, lng, name, numrat, rat, add in zip(data1.lat, data1.lng, data1.name, data1.number_of_ratings, data1.ratings, data1.address):
    folium.features.CircleMarker(
        [lat, lng],
        radius=rat*2.5 - 5,
        color=rainbow[i],
        popup="Name: {} <br> Rating: {}/10 by {} users <br> Address: {} ".format(name, rat, int(numrat), add),
        fill = True,
        fill_color=rainbow[i],
        fill_opacity=0.6
    ).add_to(venues_map)
    i += 1

# display map
venues_map

In [167]:
data2 = pd.DataFrame(columns = ['name', 'address', 'distance', 'lat', 'lng', 'id', 'number_of_ratings', 'ratings', 'from'])

for indx, ID, lat, lng in zip(data1.index, data1.id, data1.lat, data1.lng):
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&query={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, keywords[1], VERSION, radii[1], LIMIT)
    results = requests.get(url).json()
    
    dataTemp = JSON_TO_DF(results,num_of_results[1])
    dataTemp['from'] = np.ones((dataTemp.shape[0],1)) * indx
    
    data2 = pd.concat([data2, dataTemp])


In [168]:
# add the first level search results with different colors of circle markers
for i in np.arange(data1.shape[0]):
    dataTemp = data2.loc[data2['from']==i]
    for lat, lng, name, numrat, rat, add in zip(dataTemp.lat, dataTemp.lng, dataTemp.name, dataTemp.number_of_ratings, dataTemp.ratings, dataTemp.address):
        folium.features.CircleMarker(
            [lat, lng],
            radius=7,
            color=rainbow[i],
            popup="Name: {} <br> Rating: {}/10 by {} users <br> Address: {} ".format(name, rat, int(numrat), add),
            fill = True,
            fill_color=rainbow[i],
            fill_opacity=0.6
        ).add_to(venues_map)
        
venues_map

In [138]:
data2

,name,address,distance,lat,lng,id,number_of_ratings,ratings,from
0,Gartine,Taksteeg 7,162,52.369157,4.891615,4a2bbd19f964a520db961fe3,355.0,9.3,0.0
1,De Laatste Kruimel,Langebrugsteeg 4,222,52.369220,4.892810,4ee4db136c25be9630ca266e,1162.0,9.1,0.0
2,Toastable,Singel 441,68,52.367334,4.890152,504643e1e4b0a6a6519a5a42,285.0,8.9,0.0
0,Starbucks,Utrechtsestraat 9,173,52.366077,4.897431,4f019124a69d45461f2458e7,1131.0,8.4,1.0
1,Cafe Droog,Staalstraat 7B,285,52.368375,4.897672,547eff1b498e5b3cf791de96,42.0,8.4,1.0
2,Bagels & Beans,Waterlooplein 2,293,52.368850,4.900435,4a27db83f964a5203b941fe3,229.0,8.2,1.0
0,Starbucks,Utrechtsestraat 9,113,52.366077,4.897431,4f019124a69d45461f2458e7,1131.0,8.4,2.0
1,Cafe Droog,Staalstraat 7B,193,52.368375,4.897672,547eff1b498e5b3cf791de96,42.0,8.4,2.0
2,Café de Jaren,Nieuwe Doelenstraat 20-22,134,52.367956,4.895555,4a26ff58f964a5202a7f1fe3,1429.0,8.3,2.0
